참고 링크
1. https://velog.io/@sseq007/Kobert-%EB%AA%A8%EB%8D%B8-%EC%82%AC%EC%9A%A91
2. https://velog.io/@fhflwhwl5/Python-KoBERT-7%EA%B0%80%EC%A7%80-%EA%B0%90%EC%A0%95%EC%9D%98-%EB%8B%A4%EC%A4%91%EA%B0%90%EC%84%B1%EB%B6%84%EB%A5%98%EB%AA%A8%EB%8D%B8-%EA%B5%AC%ED%98%84%ED%95%98%EA%B8%B0#4-%EB%8D%B0%EC%9D%B4%ED%84%B0-%EC%A0%84%EC%B2%98%EB%A6%AC-%ED%86%A0%ED%81%B0%ED%99%94-%EC%A0%95%EC%88%98-%EC%9D%B8%EC%BD%94%EB%94%A9-%ED%8C%A8%EB%94%A9

# 모듈 임포트

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 18.1 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661677 sha256=962e9d5e2cc7689d10505fb2f08940238f2b4247dfab063c22d83760c9cdb318
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 91.6 MB/s eta 0:00:00
     ━━━━━

In [2]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-6b74kdgt/kobert-tokenizer_dd955da0c1f44a378b329a038267d70d
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-6b74kdgt/kobert-tokenizer_dd955da0c1f44a378b329a038267d70d
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=0d19a151ee38b3257ec2b460a244e02b2a6592dcc4bb0e274b7aa198ab86e067
  Stored in directory: /tmp/pip-ephem-wheel-cache-wv8leecz/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook

from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

device = torch.device("cuda:0")
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


#구글 드라이브 연동

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#데이터 로드

##데이터 로드

경로가 다를경우 에러 날 수 있으니 작업환경의 경로에 맞게 아래 코드 수정

In [5]:
train_data = pd.read_excel('/content/drive/MyDrive/toy_project/korean_an/감성대화말뭉치(최종데이터)_Training.xlsx')
valid_data = pd.read_excel('/content/drive/MyDrive/toy_project/korean_an/감성대화말뭉치(최종데이터)_Validation.xlsx')
short_data = pd.read_excel('/content/drive/MyDrive/toy_project/korean_an/한국어_단발성_대화_데이터셋.xlsx')

##데이터 확인

In [40]:
train_data.head()

,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,1,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,NaN,NaN
1,2,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,NaN,NaN
2,3,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.,NaN,NaN
3,4,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.,직장 사람들과 이야기를 해 보겠다고 결심하셨군요.,NaN,NaN
4,5,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!,항상 먼저 인사하게 되어 화가 나셨군요. 어떻게 하면 신입사원에게 화났음을 표현할 ...,NaN,NaN


In [41]:
valid_data.head()

,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,1,청년,남성,"진로,취업,직장",해당없음,불안,두려운,이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너...,실수하시다니 정말 미안한 마음이 크겠어요.,내 능력이 부족한 거 같은데 그만 다녀야 될거같아.,능력을 올리려면 어떤 방법이 있을까요?,퇴근 후 여가에 회사 일을 더 열심히 해서 피해가 가지 않도록 해야겠어.,꼭 좋은 결과 있길 바라요.
1,2,청년,남성,"진로,취업,직장",해당없음,불안,두려운,회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워.,큰 프로젝트를 혼자 하셔서 고민이 많겠네요.,나에게 너무 크게 느껴지는 중요한 프로젝트라 버거운 느낌이 들어.,프로젝트를 잘하시기 위해서 어떤 걸 할 수 있나요?,동료 직원에게 도움을 요청해서 같이 해결해야겠어.,동료 직원에게 도움을 요청하기로 하셨군요.
2,3,청년,남성,"진로,취업,직장",해당없음,불안,두려운,상사가 너무 무섭게 생겨서 친해지는 게 너무 두려워.,직장 상사가 무섭게 생기셔서 친해지는 게 어렵군요.,무섭게 생겼는데도 업무를 보려면 친해져야 할 것 같단말이야.,상사분과 친해질 수 있는 방법은 무엇이 있을까요?,먼저 다가가서 말을 걸어볼게.,직장 상사와 친해지시면 좋겠네요.
3,4,청년,남성,"진로,취업,직장",해당없음,불안,두려운,이번에 힘들게 들어간 첫 직장이거든. 첫 직장이라서 그런지 너무 긴장된다.,첫 직장이라서 정말 떨리시고 긴장되실 것 같아요.,첫 직장이어서 잘 적응을 할 수 있을지 모르겠어.,잘 적응 하시려면 무엇을 할 수 있을까요?,직장 동료와 상사들이랑 친하게 지내야겠어.,직장에 잘 적응하시길 바라요.
4,5,청년,남성,"진로,취업,직장",해당없음,불안,두려운,직장에서 동료들이랑 관계가 안 좋아질까 봐 걱정돼.,직장 사람들 관계에 대해서 고민이시군요.,내가 낯가림이 심해서 친해질 수 있을지 모르겠어.,직장 사람들과 친해지려면 무슨 방법이 있을까요?,서로 같은 취미를 공유하고 얘기를 나누다 보면 친해질 수 있을 거 같아.,직장 사람들과 좋은 관계를 가지시길 응원해요.


In [42]:
short_data.head()

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,공포
1,그냥 내 느낌일뿐겠지?,공포
2,아직너무초기라서 그런거죠?,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포


Todo
1. train_data, valid_data 필요없는 columns 제거
2. train_data, vlaid_data columns name short_data columns name으로 통일
3. Emotion 통일
4. 3개 concat(index 재정의)

In [43]:
train_data['감정_대분류'].unique()

array(['분노', '기쁨', '불안', '당황', '슬픔', '상처'], dtype=object)

In [44]:
valid_data['감정_대분류'].unique()

array(['불안', '슬픔', '당황', '기쁨', '분노', '상처'], dtype=object)

In [45]:
short_data['Emotion'].unique()

array(['공포', '놀람', '분노', '슬픔', '중립', '행복', '혐오'], dtype=object)

#데이터 클렌징

In [46]:
eda_train = pd.DataFrame()

In [47]:
eda_train['Sentence'] = train_data['사람문장1']
eda_train['Emotion'] = train_data['감정_대분류']

In [48]:
eda_train.head()

,Sentence,Emotion
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,분노
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,분노
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,분노
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,분노
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,분노


In [49]:
eda_valid = pd.DataFrame()

In [50]:
eda_valid['Sentence'] = valid_data['사람문장1']
eda_valid['Emotion'] = valid_data['감정_대분류']

In [51]:
eda_valid.head()

,Sentence,Emotion
0,이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너...,불안
1,회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워.,불안
2,상사가 너무 무섭게 생겨서 친해지는 게 너무 두려워.,불안
3,이번에 힘들게 들어간 첫 직장이거든. 첫 직장이라서 그런지 너무 긴장된다.,불안
4,직장에서 동료들이랑 관계가 안 좋아질까 봐 걱정돼.,불안


In [52]:
test_concat = pd.concat([eda_train, eda_valid, short_data], ignore_index = True)

In [53]:
test_concat.shape, eda_train.shape, eda_valid.shape, short_data.shape

((96865, 2), (51630, 2), (6641, 2), (38594, 2))

In [54]:
eda_train.shape[0] +  eda_valid.shape[0] +  short_data.shape[0] - test_concat.shape[0]

0

In [55]:
test_concat['Emotion'].unique()

array(['분노', '기쁨', '불안', '당황', '슬픔', '상처', '공포', '놀람', '중립', '행복', '혐오'],
      dtype=object)

In [56]:
test_concat.loc[(test_concat['Emotion'] == "불안"), 'Emotion'] = '공포'
test_concat.loc[(test_concat['Emotion'] == "당황"), 'Emotion'] = '놀람'
test_concat.loc[(test_concat['Emotion'] == "상처"), 'Emotion'] = '슬픔'
test_concat.loc[(test_concat['Emotion'] == "기쁨"), 'Emotion'] = '행복'

In [57]:
drop_concat = test_concat.drop(index = test_concat[test_concat['Emotion'] == '중립'].index, axis = 1)
drop_concat = drop_concat.drop(index = drop_concat[drop_concat['Emotion'] == '혐오'].index, axis = 1)

In [58]:
drop_concat['Emotion'].unique()

array(['분노', '행복', '공포', '놀람', '슬픔'], dtype=object)

In [59]:
test_concat['Emotion'].unique()

array(['분노', '행복', '공포', '놀람', '슬픔', '중립', '혐오'], dtype=object)

In [60]:
drop_concat.shape

(86606, 2)

In [61]:
drop_concat.reset_index(drop=True, inplace = True)

In [62]:
drop_concat.tail()

,Sentence,Emotion
86601,유느님 내일 이면 해피투게더가하네요~^^,행복
86602,승관아~ 너에게 편지를 안 쓸 수가 없네.,행복
86603,패떴 개그계 큰 대모 이경실 씨 1탄 잘 봤습니다,행복
86604,유재석 오라버니 해피투게더 해피파출소 편 봤어요,행복
86605,유재석 오라버니 해피투게더 봤어요,행복


In [63]:
drop_concat.loc[(drop_concat['Emotion'] == "공포"), 'Emotion'] = 0  #공포 -> 0
drop_concat.loc[(drop_concat['Emotion'] == "놀람"), 'Emotion'] = 1  #놀람 -> 1
drop_concat.loc[(drop_concat['Emotion'] == "분노"), 'Emotion'] = 2  #분노 -> 2
drop_concat.loc[(drop_concat['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 -> 3
drop_concat.loc[(drop_concat['Emotion'] == "행복"), 'Emotion'] = 4  #행복 -> 4
# drop_concat.loc[(drop_concat['Emotion'] == "중립"), 'Emotion'] = 5  #중립 -> 5

In [64]:
drop_concat['Emotion'].unique()

array([2, 4, 0, 1, 3], dtype=object)

In [65]:
data_list = []
for q, label in zip(drop_concat['Sentence'], drop_concat['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data)
print(data_list[:10])

['유재석 오라버니 해피투게더 봤어요', '4']
[['일은 왜 해도 해도 끝이 없을까? 화가 난다.', '2'], ['이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.', '2'], ['회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스트레스 받아. ', '2'], ['직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 섭섭해.', '2'], ['얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.', '2'], ['직장에 다니고 있지만 시간만 버리는 거 같아. 진지하게 진로에 대한 고민이 생겨.', '2'], ['성인인데도 진로를 아직도 못 정했다고 부모님이 노여워하셔. 나도 섭섭해.', '2'], ['퇴사한 지 얼마 안 됐지만 천천히 직장을 구해보려고.', '4'], ['졸업반이라서 취업을 생각해야 하는데 지금 너무 느긋해서 이래도 되나 싶어.', '0'], ['요즘 직장생활이 너무 편하고 좋은 것 같아!', '4']]


#데이터 분리

In [66]:
len(data_list) * 0.25

21651.5

In [67]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

print(len(dataset_train))
print(len(dataset_test))

64954
21652


#데이터 전처리

In [68]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [69]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [70]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [71]:
# BERTDataset : 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding하는 함수
tok = tokenizer.tokenize

data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

In [72]:
# torch 형식의 dataset을 만들어 입력 데이터셋의 전처리 마무리
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


#모델

In [73]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 7,   # 감정 클래스 수로 조정
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [74]:
# BERT  모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)

# optimizer와 schedule 설정
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]


optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)

# calc_accuracy : 정확도 측정을 위한 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


#학습

In [98]:
df_score = {'model' : [], 'acc' : []}

In [99]:
type(df_score['acc'])

list

In [100]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

    df_score['model'].append(model)
    df_score['acc'].append(test_acc / (batch_id + 1))

<ipython-input-100-dde897a214c2>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/1015 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch 1 batch id 1 loss 0.5936335921287537 train acc 0.75
epoch 1 batch id 201 loss 0.13386818766593933 train acc 0.8713463930348259
epoch 1 batch id 401 loss 0.3168405294418335 train acc 0.8925732543640897
epoch 1 batch id 601 loss 0.2272244095802307 train acc 0.8997764143094842
epoch 1 batch id 801 loss 0.1568310707807541 train acc 0.9001638576779026
epoch 1 batch id 1001 loss 0.4394473433494568 train acc 0.9008023226773226
epoch 1 train acc 0.900600369458128


<ipython-input-100-dde897a214c2>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/339 [00:00<?, ?it/s]

epoch 1 test acc 0.6346699852507375


  0%|          | 0/1015 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 0.4496116638183594 train acc 0.859375


KeyboardInterrupt: ignored

In [ ]:
# best_model 만들기 전의 코드

# for e in range(num_epochs):
#     train_acc = 0.0
#     test_acc = 0.0
#     model.train()
#     for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
#         optimizer.zero_grad()
#         token_ids = token_ids.long().to(device)
#         segment_ids = segment_ids.long().to(device)
#         valid_length= valid_length
#         label = label.long().to(device)
#         out = model(token_ids, valid_length, segment_ids)
#         loss = loss_fn(out, label)
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
#         optimizer.step()
#         scheduler.step()  # Update learning rate schedule
#         train_acc += calc_accuracy(out, label)
#         if batch_id % log_interval == 0:
#             print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
#     print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

#     model.eval()
#     for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
#         token_ids = token_ids.long().to(device)
#         segment_ids = segment_ids.long().to(device)
#         valid_length= valid_length
#         label = label.long().to(device)
#         out = model(token_ids, valid_length, segment_ids)
#         test_acc += calc_accuracy(out, label)
#     print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

#베스트모델 저장

In [ ]:
pd.DataFrame(df_score).sort_values('acc', ascending = False)

In [ ]:
best_model = df_score['model'][model_index] # score 내림차순 정렬해서 인덱스 확인 후 해당 인덱스 넣기

#모델 저장 및 불러오기

In [ ]:
import os

In [ ]:
my_path = '/content/drive/MyDrive/toy_project/korean_an/'

In [ ]:
torch.save(best_model.state_dict(), my_path + 'best_model.pt')

In [ ]:
best_model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)
best_model.load_state_dict(torch.load(my_path + 'best_model.pt'))
# model.eval()

#노래 추천 리스트

ToDo
1. 노래 링크 따오기
2. 검증 파트에서 나온 것에 매칭하여 랜덤으로 추출해주는 코드 작성
3. 여유되면 가수, 노래제목, 앨범커버까지 출력

감정 리스트
1. 공포 : https://blog.naver.com/tkfkd8599/90014155053
2. 놀람
3. 분노 : https://blog.naver.com/eunsilguaus/220012686691
4. 슬픔 : https://gongbuglog.tistory.com/71
5. 행복

In [ ]:
import random

In [ ]:
scary_song_list = []
suprise_song_list = []
angry_song_list = []
sad_song_list = []
happy_song_list = []

In [ ]:
scary_text_list = ['겁을 먹는 것과 까닭 없이 불안한 두려움은 확실히 구별되는 것이지만, 그러나 대부분은 단지 상상력의 기능을 한때 중단시키는 능력의 결여로 보면 된다. - 헤밍웨이',
                   '공포로 인해 타협하지 말 것이며 남이 나에게 타협하는 것을 두려워하지도 말라 - 케네디',
                   '당신을 두려워하고 있는 사람이 곧, 당신이 자리에 없을 때 당신을 혐오할 사람이다. - 토마스 풀러',
                   '쉬운 일도 신중히 하고, 곤란한 일도 겁내지 말고 해보아야 한다. 첫 고비를 두려워하기 때문에 능히 할 만한 일을 어렵다고 해서 하지 않는다. - 채근담',
                   '실패에 대한 의식적 혹은 무의식적 두려움은 당신 인생의 모든 분야에서 꿈을 이루려는 당신의 능력을 방해한다. - 스티븐 스코트']
suprise_text_list = ['']
angry_text_list = ['깊은 강물은 돌을 던져도 흐리지 않는다. 모욕을 받고 이내 격분하는 사람은 강이 아닌 조그마한 웅덩이에 불과하다. -톨스토이',
                   '늘 쾌활하게 생활하고 싶다면 사소한 일에 화를 내지 말 것이며, 비록 작더라도 제 몫으로 온 것에 대해서 만족하고 감사히 여겨라. -사무엘 스마일즈',
                   '분노는 기묘한 사용법을 지닌 무기다. 다른 무기는 사람이 사용하지만 분노라는 무기는 반대로 우리를 사용한다. -몽테뉴',
                   '분노를 떨쳐버리려면 아무 것도 하지 않는 것이 최선의 방법이다. -세네카',
                   '화가 난 사람은 장님과 바보가 된다. 이성은 사라져 버리고 노여움은 지성의 힘을 완전히 억누르며, 판단력도 그것의 포로가 되어 모든 기능은 완전히 멎기 때문이다. -피에트로 아레티노']
sad_text_list = ['슬픔의 유일한 치유제는 행동이다. - 조지 헨리 루이스',
                 '지나간 슬픔에 새로운 눈물을 낭비하지 마라. - 에우리피데스',
                 '슬픔은 한결같은 사람에게 흔들림을 가르친다. - 소포클레스',
                 '자신이 공들이고 견뎌낸 모든 것을 기억하는 사람에게는 슬픔조차도 오랜 시간이 지나면 기쁨이 된다. - 호메로스',
                 '행복은 몸에 좋다. 그러나 마음의 힘을 길러주는 것은 슬픔이다. - 마르쉘 프루스트']
happy_text_list = ['']

#검증

In [ ]:
# 원본 코드

# def predict(predict_sentence): # input = 감정분류하고자 하는 sentence

#     data = [predict_sentence, '0']
#     dataset_another = [data]

#     another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False) # 토큰화한 문장
#     test_dataloader = torch.utils.data.DataLoader(another_test, batch_size = batch_size, num_workers = 2) # torch 형식 변환

#     model.eval()

#     for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
#         token_ids = token_ids.long().to(device)
#         segment_ids = segment_ids.long().to(device)

#         valid_length = valid_length
#         label = label.long().to(device)

#         out = model(token_ids, valid_length, segment_ids)


#         test_eval = []
#         for i in out: # out = model(token_ids, valid_length, segment_ids)
#             logits = i
#             logits = logits.detach().cpu().numpy()

#             if np.argmax(logits) == 0:
#                 test_eval.append("공포가")
#             elif np.argmax(logits) == 1:
#                 test_eval.append("놀람이")
#             elif np.argmax(logits) == 2:
#                 test_eval.append("분노가")
#             elif np.argmax(logits) == 3:
#                 test_eval.append("슬픔이")
#             elif np.argmax(logits) == 4:
#                 test_eval.append("행복이")

#         print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [ ]:
# 원본 코드

# # 질문에 0 입력 시 종료
# end = 1
# while end == 1 :
#     sentence = input("하고싶은 말을 입력해주세요 : ")
#     if sentence == "0" :
#         break
#     predict(sentence)
#     print("\n")

In [ ]:
def predict_v2(predict_sentence): # input = 감정분류하고자 하는 sentence

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False) # 토큰화한 문장
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size = batch_size, num_workers = 2) # torch 형식 변환

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length = valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval = []
        # 모델이 판단한 감정 출력
        for i in out: # out = model(token_ids, valid_length, segment_ids)
            logits = i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("공포가")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("행복이")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

        # 모델이 판정한 감정에 어울리는 문구 출력
        for i in out: # out = model(token_ids, valid_length, segment_ids)
            logits = i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                print(random.sample(scary_text_list, 1))
            elif np.argmax(logits) == 1:
                print(random.sample(suprise_text_list, 1))
            elif np.argmax(logits) == 2:
                print(random.sample(angry_text_list, 1))
            elif np.argmax(logits) == 3:
                print(f"{random.sample(sad_text_list, 1)}")
            elif np.argmax(logits) == 4:
                print(random.sample(happy_text_list, 1))

        # 모델이 판정한 감정에 어울리는 노래 3곡 추천
        # 리스트 내에 3개 이상의 데이터가 없어서 에러가 발생하여 잠시 주석처리
        # for i in out: # out = model(token_ids, valid_length, segment_ids)
        #     logits = i
        #     logits = logits.detach().cpu().numpy()

        #     if np.argmax(logits) == 0:
        #         print(random.sample(scary_song_list, 3))
        #     elif np.argmax(logits) == 1:
        #         print(random.sample(suprise_song_list, 3))
        #     elif np.argmax(logits) == 2:
        #         print(random.sample(angry_song_list, 3))
        #     elif np.argmax(logits) == 3:
        #         print(random.sample(sad_song_list, 3))
        #     elif np.argmax(logits) == 4:
        #         print(random.sample(happy_song_list, 3))


In [ ]:
# 질문에 0 입력 시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict_v2(sentence)
    print("\n")

#저장된 모델 확인

나중에 이 섹션 지우고 검증파트의 모델을 best_model 적용하는 함수 적용

In [ ]:
def best_predict(predict_sentence): # input = 감정분류하고자 하는 sentence

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False) # 토큰화한 문장
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size = batch_size, num_workers = 2) # torch 형식 변환

    best_model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length = valid_length
        label = label.long().to(device)

        out = best_model(token_ids, valid_length, segment_ids)


        test_eval = []
        for i in out: # out = model(token_ids, valid_length, segment_ids)
            logits = i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("공포가")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("행복이")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

        # 모델이 판정한 감정에 어울리는 문구 출력
        for i in out: # out = model(token_ids, valid_length, segment_ids)
            logits = i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                print(f"{random.sample(scary_text_list, 1)}")
            elif np.argmax(logits) == 1:
                print(random.sample(suprise_text_list, 1))
            elif np.argmax(logits) == 2:
                print(random.sample(angry_text_list, 1))
            elif np.argmax(logits) == 3:
                print(f"{random.sample(sad_text_list, 1)}")
            elif np.argmax(logits) == 4:
                print(random.sample(happy_text_list, 1))

        # 모델이 판정한 감정에 어울리는 노래 3곡 추천
        # 리스트 내에 3개 이상의 데이터가 없어서 에러가 발생하여 잠시 주석처리
        # for i in out: # out = model(token_ids, valid_length, segment_ids)
        #     logits = i
        #     logits = logits.detach().cpu().numpy()

        #     if np.argmax(logits) == 0:
        #         print(random.sample(scary_song_list, 3))
        #     elif np.argmax(logits) == 1:
        #         print(random.sample(suprise_song_list, 3))
        #     elif np.argmax(logits) == 2:
        #         print(random.sample(angry_song_list, 3))
        #     elif np.argmax(logits) == 3:
        #         print(random.sample(sad_song_list, 3))
        #     elif np.argmax(logits) == 4:
        #         print(random.sample(happy_song_list, 3))


In [ ]:
# 질문에 0 입력 시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    best_predict(sentence)
    print("\n")